In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from albumentations import Compose, RandomBrightnessContrast, GaussNoise, Blur


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# in case we need them
from albumentations.pytorch import ToTensorV2

# Core layers
from keras.layers \
    import Activation, Dropout, Flatten, Dense, Input, LeakyReLU

# Normalization layers
from keras.layers import BatchNormalization

# Merge layers
from keras.layers import concatenate, multiply

# Embedding Layers
from keras.layers import Embedding

# Keras models
from keras.models import Model, Sequential

# Keras optimizers
from tensorflow.keras.optimizers import Adam, RMSprop, SGD


## Augmentations

In [ ]:
augmentations = Compose([
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
    GaussNoise(var_limit=(10, 50)),
    Blur(blur_limit=3)
])
augmentations

In [ ]:
train_gan_path = "/kaggle/input/mjsynth/mnt/ramdisk/max/90kDICT32px" # change path if needed

In [ ]:
import glob
import os
def preprocess_image(image):
    # Resize, normalize, and other preprocessing steps
    image = tf.image.resize(image, (32, 128))
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return image
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (32, 128))
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

image_directory = "/kaggle/input/mjsynth/mnt/ramdisk/max/90kDICT32px/1015/4"  # contains 501 images (of size (32, 128, 3))
image_file_paths = glob.glob(os.path.join(image_directory, "*.jpg"))  
image_dataset = tf.data.Dataset.from_tensor_slices(image_file_paths)
image_dataset = image_dataset.map(load_and_preprocess_image)

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    
    # Foundation for 4x16 image
    model.add(layers.Dense(256 * 4 * 16, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((4, 16, 256)))

    # Upsample to 8x32
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 16x64
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 32x128
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 128, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

Check these!

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
discriminator = make_discriminator_model()
generator = make_generator_model()
import numpy as np

noise_dim = 100  # This should match the input shape of the generator
random_noise = np.random.normal(0, 1, size=(1, noise_dim))
batch_size = 32  # Set your desired batch size
image_dataset = image_dataset.batch(batch_size)

In [ ]:

def train_step(images, generator, discriminator, generator_optimizer, discriminator_optimizer, batch_size, noise_dim):
    """
    A single training step for the GAN.
    """
    disri_losses = []
    gener_losses = []
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    #print(f"epoch = {epoch}, Generator Loss: {gen_loss.numpy()}, Discriminator Loss: {disc_loss.numpy()}")
    discri_losses.append(disc_loss.numpy())
    gener_losses.append(gen_loss.numpy())

print(f"epoch = {epoch}, Generator Loss: {gen_loss.numpy()}, Discriminator Loss: {disc_loss.numpy()})   
return discri_losses, gener_losses 

# Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Example usage within a training loop
epochs = 500
for epoch in range(epochs):
    for image_batch in image_dataset:
        d, g = train_step(image_batch, generator, discriminator, generator_optimizer, discriminator_optimizer, batch_size, noise_dim)
        

In [ ]:
# discriminator loss
d[-1]

In [ ]:
# generator loss
g[-1]

In [ ]:
generated_image = generator.predict(random_noise)
import matplotlib.pyplot as plt

def visualize_image(generated_image):
    plt.imshow((generated_image[0, :, :, :] * 127.5 + 127.5).astype('uint8'))
    plt.axis('off')
    plt.show()

visualize_image(generated_image)

i